<img src="images/example_images.png" style="width:600px;"/>

$$
\begin{aligned}
&|\psi\rangle \rightarrow |0010\rangle|0110\rangle|1010\rangle|0101\rangle &\rightarrow \frac{1}{\sqrt{4}}(0010011000100000)^T\\
&|\phi_0\rangle \rightarrow |0010\rangle|0110\rangle|1010\rangle &\rightarrow \frac{1}{\sqrt{3}}(0010001000100000)^T\\
&|\phi_1\rangle \rightarrow |0110\rangle|1010\rangle|0001\rangle &\rightarrow \frac{1}{\sqrt{3}}(0100001000100000)^T\\
&|\phi_2\rangle \rightarrow |0000\rangle|0100\rangle|1000\rangle &\rightarrow \frac{1}{\sqrt{3}}(1000100010000000)^T\\
&|\phi_3\rangle \rightarrow |0000\rangle|0100\rangle|1000\rangle|1001\rangle &\rightarrow \frac{1}{\sqrt{4}}(1000100011000000)^T
\end{aligned}
$$

# Oracle W
$$
j = \text{Index of training image}\\
|0^n\rangle \xrightarrow{\mathcal{V}}|\psi\rangle\\
|j\rangle|0^n\rangle\xrightarrow{\mathcal{W}}|j\rangle|\phi_j\rangle
$$

Example:

$$
|0^n\rangle \xrightarrow{\mathcal{V}}\frac{1}{\sqrt{11}}|1231\rangle\\
|00\rangle|0^n\rangle\xrightarrow{\mathcal{W}}|00\rangle\frac{1}{\sqrt{11}}|1130\rangle
$$

Variables:

$$
\begin{aligned}
M &= \text{Amount of train images}\\
N &= \text{Dimension of Hilbert space}\\
m &= \log{M}\\
n &= \log{N}\\
b &= \text{Number of qubits required to store the fidelity value}
\end{aligned}
$$

## Circuits:
### Circuit complete
<img src="images/circuit_complete.png" style="width:800px;"/>

### Circuit Eamp
<img src="images/circuit_eamp.png" style="width:600px;"/>


In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, IBMQ, \
                   Aer, BasicAer, assemble, transpile, execute
from qiskit.circuit.library.standard_gates.swap import SwapGate
from qiskit.providers.ibmq import least_busy
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.extensions.quantum_initializer.initializer import Initialize
from qiskit.aqua import QuantumInstance
from qiskit.quantum_info.operators import Operator
from qiskit_textbook.tools import array_to_latex

from numpy import pi
import numpy as np
import math

# Custom
from util import *

/Users/robinbux/anaconda3/lib/python3.8/site-packages/qiskit/quantum_info/operators/pauli.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z = np.zeros(len(label), dtype=np.bool)
/Users/robinbux/anaconda3/lib/python3.8/site-packages/qiskit/quantum_info/operators/pauli.py:112: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros(len(label

ModuleNotFoundError: No module named 'qiskit_textbook'

In [ ]:
# Variables
m = 2
n = 4

In [ ]:
W_oracle = create_w_oracle(j = 0)
V_oracle = create_v_oracle()
U_oracle = create_u_oracle()

index_register = QuantumRegister(2, 'index')
train_register = QuantumRegister(4, 'train')
test_register = QuantumRegister(4, 'test')
B_register = QuantumRegister(1, 'b')
qc = QuantumCircuit(index_register, train_register, test_register, B_register)

qc.append(W_oracle, [*index_register[:],*train_register[:]])
qc.append(U_oracle, [*train_register[:],*test_register[:],B_register[0]])

qc.decompose().draw(output="mpl", initial_state=True)

In [ ]:
#def swap_test(phi_state, psi_state):
sv = Statevector.from_label('111111111')
qc = QuantumCircuit(n*2 + 1)
sv.evolve(qc)

circuit = QuantumCircuit(6)
circuit.append(Initialize("1+-"), range(3))
circuit.append(Initialize("101"), list(np.arange(3,6)))

#for qubit in range (n):
#    circuit.cswap(B_register[0], train_register[qubit], test_register[qubit])
#qc.h(B_register[0])
circuit.decompose().draw()

In [ ]:
def swap_test(psi_state, phi_state):
    n = len(psi_state)
    qc = QuantumCircuit(n*2 + 1, 1)
    qc.append(Initialize(psi_state), range(n))
    qc.append(Initialize(phi_state), list(np.arange(n,n*2)))
    qc.h(n*2)

    for qubit in range (n):
        qc.cswap(n*2, qubit, n+qubit)
    
    qc.h(n*2)
    qc.measure(n*2, 0)
    return qc

In [ ]:
psi_state_one = "000100010"
phi_state_one = "000010000"

psi_state_two = "111111111"
phi_state_two = "111111111"

qc = swap_test(psi_state_two, phi_state_two)
#qc.draw(output="mpl")

simulator = BasicAer.get_backend('qasm_simulator')

job = execute(qc, simulator, shots = 1000)

result = job.result()
counts = result.get_counts(qc)

print(counts)

#print(result)

In [ ]:
qc.draw(output="mpl")

In [ ]:
psi = np.array([[0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0]])
phi_zero = np.array([[0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0]])
phi_one = np.array([[0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0]])
phi_two = np.array([[1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0]])
phi_three = np.array([[1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0]])

matrix_psi = create_unitary(psi)
matrix_phi_zero = create_unitary(phi_zero)
matrix_phi_one = create_unitary(phi_one)
matrix_phi_two = create_unitary(phi_two)
matrix_phi_three = create_unitary(phi_three)

In [ ]:
controls = QuantumRegister(4)
circuit = QuantumCircuit(controls)

cx = Operator(matrix_psi)
circuit.unitary(cx, [0, 1, 2, 3], label='cx')
circuit.draw()

In [ ]:
cx.is_unitary()

In [ ]:
m = 2
n = 4

psi = np.array([[0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0]])
phi_zero = np.array([[0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0]])
phi_one = np.array([[0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0]])
phi_two = np.array([[1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0]])
phi_three = np.array([[1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0]])

matrix_psi = create_unitary(psi)
matrix_phi_zero = create_unitary(phi_zero)
matrix_phi_one = create_unitary(phi_one)
matrix_phi_two = create_unitary(phi_two)
matrix_phi_three = create_unitary(phi_three)

phi_matrices = [matrix_phi_zero, matrix_phi_one, matrix_phi_two, matrix_phi_three]


In [ ]:
def test(j):
    W_state_preparation_oracle_circuit = QuantumCircuit(m + n)
    
    matrix_operator = Operator(phi_matrices[j])
    # [2,3,4,5]
    # list(range(n, n+m))
    W_state_preparation_oracle_circuit.unitary(matrix_operator, np.arange(m,m+n), label='W')
    
    W_oracle = W_state_preparation_oracle_circuit.to_gate()
    W_oracle.name = "$W$"
    return W_oracle

In [ ]:
test(0)

In [ ]:
zero_ket = np.zeros((2**9, 1))
zero_ket[0] = 1
zero_bra = zero_ket.T
np.outer(zero_ket, zero_bra)

In [ ]:
zero_ket = np.zeros((2, 1))
zero_ket[0] = 1
zero_bra = zero_ket.T

In [ ]:
test = np.outer(zero_ket, zero_bra)
test[0][1] = 1
test.T

In [ ]:
j = 0
W_oracle = create_w_oracle(j)
U_oracle = create_u_oracle()
G_gate = create_g_gate(j)
    
index_register = QuantumRegister(2, 'index')
train_register = QuantumRegister(4, 'train')
test_register = QuantumRegister(4, 'test')
B_register = QuantumRegister(1, 'b')
qc = QuantumCircuit(index_register, train_register, test_register, B_register)


qc.append(W_oracle, [*index_register[:],*train_register[:]])
qc.append(U_oracle, [*test_register[:],*train_register[:], B_register[0]])
qc.append(G_gate, [*index_register[:],*train_register[:], *test_register[:], B_register[0]])

qc.decompose().draw(output="mpl", initial_state=True)

In [ ]:
# phase_est_gate = phase_estimation_on_g(0)

# index_register = QuantumRegister(2, 'index')
# train_register = QuantumRegister(4, 'train')
# test_register = QuantumRegister(4, 'test')
# B_register = QuantumRegister(1, 'b')
# phase_register = QuantumRegister(4, 'phase')
    
# qc = QuantumCircuit(index_register, train_register, test_register, B_register, phase_register)
# qc.append(phase_est_gate, [*index_register[:],*train_register[:], *test_register[:], B_register[0], *phase_register[:]])

# qc.draw(output="mpl", initial_state=True)


In [ ]:
# Variables
m = 2
n = 4
b = 3

In [ ]:
def phase_estimation_on_g(j):
    G_gate_controll = create_g_gate(j).control(1)
    
    index_register = QuantumRegister(m, 'index')
    train_register = QuantumRegister(n, 'train')
    test_register = QuantumRegister(n, 'test')
    B_register = QuantumRegister(1, 'b')
    phase_register = QuantumRegister(b, 'phase')    
    
    phase_est_circ = QuantumCircuit(index_register, train_register, test_register, B_register, phase_register)
    
    # Hadamard
    for qubit in range(b):
        phase_est_circ.h(phase_register[qubit])

    for phase_index in range(b):
        for gate_index in range(2**b):
            phase_est_circ.append(G_gate_controll, [*index_register[:],*train_register[:], *test_register[:], B_register[0], phase_register[phase_index]])

    phase_est_gate = phase_est_circ.to_gate()
    phase_est_gate.name = "$PhaseEstimation on G$"
    return phase_est_gate

In [ ]:
phase_est_gate = phase_estimation_on_g(0)

index_register = QuantumRegister(2, 'index')
train_register = QuantumRegister(4, 'train')
test_register = QuantumRegister(4, 'test')
B_register = QuantumRegister(1, 'b')
phase_register = QuantumRegister(3, 'phase')
    
qc = QuantumCircuit(index_register, train_register, test_register, B_register, phase_register)
qc.append(phase_est_gate, [*index_register[:],*train_register[:], *test_register[:], B_register[0], *phase_register[:]])

qc.decompose().draw(output="mpl", initial_state=True)